<a href="https://colab.research.google.com/github/amina04/CNN-lstm/blob/main/ML_SVM_cicids2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd  
import tensorflow as tf

In [ ]:
d1=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
d2=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')
d3=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Friday-WorkingHours-Morning.pcap_ISCX.csv')
d4=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Monday-WorkingHours.pcap_ISCX.csv')
d5=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')
d6=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
d7=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Wednesday-workingHours.pcap_ISCX.csv')
d8=pd.read_csv('/content/drive/MyDrive/data/cicids-2017/Tuesday-WorkingHours.pcap_ISCX.csv')


In [ ]:
data=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8])
data.shape

(2830743, 79)

In [ ]:
data=pd.concat([d1,d2,d3,d4,d5,d6,d7,d8]).drop_duplicates(keep=False)
data.reset_index(drop=True, inplace = True)

In [ ]:
del d1,d2,d3,d4,d5,d6,d7

In [ ]:
data.shape

(2427193, 79)

In [ ]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2427193 entries, 0 to 2427192
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [ ]:
data.columns
print(data[' Label'].unique())
len(data[' Label'].unique())

['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'DoS slowloris' 'DoS Slowhttptest'
 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' 'FTP-Patator' 'SSH-Patator']


15

**Data Cleaning**
This chapter contains data cleaning code. We go through the process of renaming columns, removing NaN and non-finite values (-inf, inf) to get the data ready for visualization and model training.

In [ ]:
# Removing whitespaces in column names.
#renommer les colonnes
col_names = [col.replace(' ', '') for col in data.columns]
data.columns = col_names
data.head()

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,min_seg_size_forward,ActiveMean,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [ ]:
data.columns
print(data['Label'].unique())
len(data['Label'].unique())

['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'DoS slowloris' 'DoS Slowhttptest'
 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' 'FTP-Patator' 'SSH-Patator']


15

In [ ]:
# Here we can see that 'Label' column contains some wierd characters. 

data["Label"].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed', 'FTP-Patator',
       'SSH-Patator'], dtype=object)

In [ ]:
# This next snippet uses regular expressions to replace wierd characters with dunders.

label_names = data['Label'].unique()


import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

(['BENIGN',
  'DDoS',
  'PortScan',
  'Bot',
  'Infiltration',
  'Web_Attack_Brute_Force',
  'Web_Attack_XSS',
  'Web_Attack_Sql_Injection',
  'DoS_slowloris',
  'DoS_Slowhttptest',
  'DoS_Hulk',
  'DoS_GoldenEye',
  'Heartbleed',
  'FTPPatator',
  'SSHPatator'],
 15)

In [ ]:
# Replacing 'Label' column values with new readable values.

labels = data['Label'].unique()

for i in range(0,len(label_names)):
    data['Label'] = data['Label'].replace({labels[i] : label_names[i]})  
data['Label'].unique()

array(['BENIGN', 'DDoS', 'PortScan', 'Bot', 'Infiltration',
       'Web_Attack_Brute_Force', 'Web_Attack_XSS',
       'Web_Attack_Sql_Injection', 'DoS_slowloris', 'DoS_Slowhttptest',
       'DoS_Hulk', 'DoS_GoldenEye', 'Heartbleed', 'FTPPatator',
       'SSHPatator'], dtype=object)

In [ ]:
# Checking if there are any NULL values in the dataset.

data.isnull().values.any()

True

In [ ]:
# Checking which column/s contain NULL values.

[col for col in data if data[col].isnull().values.any()]

['FlowBytes/s']

In [ ]:
# Checking how many NULL values it this column contains.

data['FlowBytes/s'].isnull().sum()

334

In [ ]:
# Removing rows that contain NULL values and checking if number of removed rows is equal to the number of null values.

before = data.shape
data.dropna(inplace=True)
after = data.shape
before[0] - after[0]

334

In [ ]:
data.isnull().any().any()

False

**Removing non-finite values**

In [ ]:
import numpy as np

In [ ]:
labl = data['Label']
data = data.loc[:, data.columns != 'Label'].astype('float64')

In [ ]:
# Checking if all values are finite.
np.all(np.isfinite(data))

False

In [ ]:
# Checking what column/s contain non-finite values.

nonfinite = [col for col in data if not np.all(np.isfinite(data[col]))]
nonfinite

['FlowBytes/s', 'FlowPackets/s']

In [ ]:
# Checking how many non-finite values each column contains.

finite= np.isfinite(data['FlowBytes/s']).sum()
data.shape[0] - finite

1132

In [ ]:
# Checking how many non-finite values each column contains.

finite= np.isfinite(data['FlowPackets/s']).sum()
data.shape[0] - finite

1132

In [ ]:
# Same as before, since there is a small number of non-finite values we can safely remove them from the dataset
# without spoiling the dataset.

# Replacing infinite values with NaN values.
data = data.replace([np.inf,-np.inf],np.nan)

In [ ]:
# We can see that now we have Nan values again.

np.any(np.isnan(data))

True

In [ ]:
# Bringing the Labels back into the dataset before deliting Nan rows.

data = data.merge(labl, how='outer', left_index=True, right_index=True)

In [ ]:
# Removing new NaN values.

before = data.shape
data.dropna(inplace=True)
after = data.shape
before[0] - after[0]

1132

In [ ]:
data.shape

(2425727, 79)

**Data preparation**

In [ ]:
# Splitting dataset into features and labels.

labels = data['Label']
features = data.loc[:, data.columns != 'Label'].astype('float64')

In [ ]:
labels.head()
i=0
for row in labels:
  if row!='BENIGN':
    i+=1
print(i)

390222


In [ ]:
# For scaling the data, we use RobustScaler class from sklearn.

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features = scaler.fit_transform(features)


In [ ]:
# Checking if scaling has been succesful.
features[0]

array([8.37186236e-01, 1.33333321e-07, 4.55046005e-06, 0.00000000e+00,
       9.30232558e-07, 0.00000000e+00, 2.41740532e-04, 2.58064516e-03,
       1.00995527e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.13636364e-01, 4.44444444e-01,
       1.33333319e-07, 0.00000000e+00, 1.33333319e-07, 1.41666650e-07,
       2.50000000e-08, 2.50000000e-08, 0.00000000e+00, 2.50000000e-08,
       1.24999988e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 9.99855825e-01, 9.94591933e-01,
       2.22222222e-01, 0.00000000e+00, 4.14364641e-03, 2.41740532e-04,
       1.79794521e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.31164384e-03, 1.00995527e-03, 0.00000000e+00, 9.99855825e-01,
      

**Label encoding**

In [ ]:
#multiclass
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
LE.fit(labels)
labels = LE.transform(labels)
# Labels have been replaced with integers.
np.unique(labels)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [ ]:
#binary
for i in range(len(labels)):
 if labels[i]==0:
   labels[i]=0
 else :
   labels[i]=1
np.unique(labels)
# labels=pd.get_dummies(labels)
# labels = scaler.fit_transform(labels)
labels[0]

0

**Splitting the data**
Final step to data preparation is splitting the data into traning and testing sets. For this there already exists sklearn function that does all the splitting for us. This step is important so we can have representative data for evaluating our model. Both train and test samples should contain similar data variance.

In [ ]:
from sklearn.model_selection import train_test_split
# The next step is to split training and testing data. For this we will use sklearn function train_test_split().
# In the first step we will split the data in training and remaining dataset
x_train, x_test, y_train, y_test = train_test_split(features,labels, train_size=0.8)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1940581, 78), (485146, 78), (1940581,), (485146,))

In [ ]:
#imports
import numpy as np
from scipy.io import loadmat
from sklearn import svm


In [ ]:
svc = svm.SVC()
# par default of svc C=1.0, kernel='rbf', degree=3, gamma='scale' 3 for avoid UF or OF
svc.fit(x_train, y_train)

# Testing
#print('Test accuracy = {0}%'.format(np.round(svc.score(x_test, y_test) * 100, 2)))


In [ ]:
pred = svc.predict(x_test)
#check accuaracy of prediction
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score, recall_score
#model.score(x_valid,y_valid)

print('Confusion Matrix')
cf_matrix=confusion_matrix(y_test,pred)
print(cf_matrix)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(cf_matrix, cmap='coolwarm',annot=True, linewidth=1,fmt="d")
plt.show()

In [ ]:
recall = recall_score(y_test, pred , average="binary")
precision = precision_score(y_test, pred, average="binary")
FPR = cf_matrix[0][1]/(cf_matrix[0][1]+cf_matrix[1][1]) 

In [ ]:
print("Precision : " , precision*100)
print("Recall : ", recall*100)
print("Accuracy : ",accuracy)
print("FPR : ",FPR*100)